In [5]:
import Pkg
Pkg.activate(".")

  Activating new project at `C:\Users\alexm\Downloads`


In [7]:
Pkg.add(["Random", "Images", "Colors", "LinearAlgebra", "MLDatasets", "MultivariateStats", "Plots", "Convex", "SCS"])

    Updating registry at `C:\Users\alexm\.julia\registries\General.toml`
   Resolving package versions...
   Installed ShowCases ──────────────────────── v0.1.0
   Installed GR_jll ─────────────────────────── v0.72.8+0
   Installed HypergeometricFunctions ────────── v0.3.25
   Installed TiledIteration ─────────────────── v0.5.0
   Installed ImageIO ────────────────────────── v0.6.9
   Installed MutableArithmetics ─────────────── v1.6.0
   Installed ImageSegmentation ──────────────── v1.8.2
   Installed TiffImages ─────────────────────── v0.11.1
   Installed ContextVariablesX ──────────────── v0.1.3
   Installed ImageMagick ────────────────────── v1.4.0
   Installed LoggingExtras ──────────────────── v1.1.0
   Installed HistogramThresholding ──────────── v0.3.1
   Installed RegionTrees ────────────────────── v0.3.2
   Installed NNlib ──────────────────────────── v0.9.24
   Installed ZipFile ────────────────────────── v0.10.1
   Installed InlineStrings ──────────────────── v1.4.2
   Inst

In [ ]:
using Images, Colors, MLDatasets, LinearAlgebra, MultivariateStats, Plots, Convex, SCS

In [ ]:
dataset = MNIST()

In [ ]:
testset = MNIST(:test)

In [ ]:
dataset.targets

In [ ]:
dataset.features

In [ ]:
Gray.(dataset.features[:,:,5]')

## BME 574 Midterm Exam (take-home)

The dataset above is the famous MNIST dataset of handwritten digits (28x28 grayscale images).
The challenge for this midterm is to find the most informative pixels for each digit from this dataset
using techniques that you have learning previously.  Here is how to start:
1. Create a set of output (label) vectors $\{y_j\}$ of length 10 that represent the labels (0-9)
here are a few examples:
$$
\begin{align}
^{"}1^{"}=\begin{bmatrix} 1\\0\\0\\ \vdots \\0 \end{bmatrix}
\end{align}
$$
$$
\begin{align}
^{"}3^{"}=\begin{bmatrix} 0\\0\\1\\ \vdots \\0 \end{bmatrix}
\end{align}
$$
$$
\begin{align}
^{"}0^{"}=\begin{bmatrix} 0\\0\\0\\ \vdots \\1 \end{bmatrix}
\end{align}
$$
2. Now let $B$ be the set of output (label) vectors

$$
\begin{align}
B = \begin{bmatrix} y_{1} & y_{2} & y_{3} & \ldots & y_{n}\end{bmatrix}
\end{align}
$$

3. Let the matrix $A$ be the corresponding reshaped (vectorized) MNIST images (one in each row)

$$
\begin{align}
A = \begin{bmatrix} x_{1} & x_{2} & x_{3} & \ldots & x_{n}\end{bmatrix}
\end{align}
$$

4. $AX = B$ maps from the image space to the label space

Your task is to use solvers for $AX=B$ that promote sparsity to rank the pixels in the MNIST dataset that are most informative for correctly labeling the digits. You will have to come up with your own heuristics or empirical rules for this. Be sure to visualize the results from X. Apply your most important pixels to the test data set to see how accurate you are with as few pixels as possible. Redo the analysis with each digit individually to ﬁnd the most important pixels for each digit. Think about the interpretation of what you are doing with this $AX = B$ problem.
Hint: for testing, it may be a good idea to start with a smaller dataset (e.g. 1000 images)

In [ ]:
# Use a smaller subset of MNIST for computational efficiency
train_images = Float64.(dataset.features[:, :, 1:1000])  # First 1000 training images
train_labels = dataset.targets[1:1000]                  # Corresponding labels

test_images = Float64.(testset.features[:, :, 1:1000])  # First 1000 test images
test_labels = testset.targets[1:1000]                  # Corresponding labels

In [ ]:
# Flatten MNIST training images into a matrix (each column is a flattened image)
A = hcat([vec(train_images[:, :, i]) for i in 1:1000]...)

# MNIST training labels as a vector
B = train_labels

In [ ]:
function lasso(A, y, λ1)
    (T, K) = (size(A, 1), size(A, 2))
    Q = A'A / T
    c = A'y / T

    x = Variable(K)                  # Optimization variable
    L1 = quadform(x, Q; assume_psd=true)  # Quadratic term
    L2 = dot(c, x)                     # Linear term
    L3 = norm(x, 1)                    # Sparsity-promoting term

    problem = minimize(L1 - 2 * L2 + λ1 * L3)
    solve!(problem, SCS.Optimizer; silent=true)
    return vec(evaluate(x))            # Return the optimized weights
end

In [ ]:
lambdas = [0.1, 0.5, 1.0, 5.0, 10.0]
lasso_results = Dict()
accuracies = []

for λ in lambdas
    # Apply Lasso regression
    weights = lasso(A', B, λ)
    important_pixels = findall(x -> abs(x) > 1e-4, weights)

    # Reduce MNIST data to selected sparse pixels
    A_reduced = A[important_pixels, :]
    test_A_reduced = hcat([vec(test_images[:, :, i])[important_pixels] for i in 1:1000]...)

    # Solve AX = B using reduced data
    X = pinv(A_reduced) * B
    test_predictions = round.(test_A_reduced' * X)
    test_accuracy = sum(test_predictions .== test_labels) / length(test_labels)

    # Store results for each lambda
    lasso_results[λ] = (important_pixels, test_accuracy)
    push!(accuracies, test_accuracy)
end

# Find the best lambda based on test accuracy
best_lambda_index = argmax(accuracies)
best_lambda = lambdas[best_lambda_index]
(important_pixels, test_acc) = lasso_results[best_lambda]
println("Best λ: ", best_lambda)
println("Test Accuracy: ", test_acc)

In [ ]:
# Visualize the most informative MNIST pixels
pixel_image = zeros(size(A, 1))
pixel_image[important_pixels] .= 1.0
heatmap(reshape(pixel_image, 28, 28), color=:viridis, title="Top Pixels (Lasso, λ = $best_lambda)")

In [ ]:
digit_pixels = Dict{Int, Vector{Int}}()
accuracy_drop = Dict()

for digit in 0:9
    # Filter MNIST data for the current digit
    digit_indices = findall(x -> x == digit, B)
    A_digit = A[:, digit_indices]
    B_digit = B[digit_indices]

    # Apply Lasso for digit-specific analysis
    weights_digit = lasso(A_digit', B_digit, best_lambda)
    important_pixels_digit = findall(x -> abs(x) > 1e-4, weights_digit)
    digit_pixels[digit] = important_pixels_digit

    # Evaluate test accuracy with sparse features for the digit
    A_reduced_digit = A[important_pixels_digit, :]
    test_A_reduced_digit = hcat([vec(test_images[:, :, i])[important_pixels_digit] for i in 1:1000]...)
    X_digit = pinv(A_reduced_digit) * B
    test_predictions_digit = round.(test_A_reduced_digit' * X_digit)
    digit_accuracy = sum(test_predictions_digit .== test_labels) / length(test_labels)

    accuracy_drop[digit] = digit_accuracy
    println("Digit $digit Accuracy: ", digit_accuracy)
end

In [ ]:
function custom_metric(predictions)
    confidence_scores = map(x -> sum(predictions .== x) / length(predictions), 0:9)
    most_likely_digit = argmax(confidence_scores)
    return most_likely_digit, confidence_scores
end

# Apply the custom metric to MNIST test predictions
most_likely, scores = custom_metric(test_predictions)
println("Most Likely Digit: ", most_likely)
println("Confidence Scores: ", scores)